In [1]:
from kafka import KafkaProducer
import time

producer = KafkaProducer(
    bootstrap_servers='namenode:9092',
    value_serializer=lambda x: str(x).encode('utf-8')
)

for i in range(10):
    message = f"Test message {i}"
    producer.send('test-topic', value=message)
    print(f"Sent: {message}")
    time.sleep(1)

producer.flush()

Sent: Test message 0
Sent: Test message 1
Sent: Test message 2
Sent: Test message 3
Sent: Test message 4
Sent: Test message 5
Sent: Test message 6
Sent: Test message 7
Sent: Test message 8
Sent: Test message 9


In [5]:
from kafka import KafkaConsumer
import json
c = KafkaConsumer('test-topic', bootstrap_servers=['namenode:9092'])


In [6]:
def process_msg(m):
    print(m)

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("KafkaSparkIntegration") \
    .master("local[*]") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1") \
    .config("spark.sql.streaming.checkpointLocation", "/tmp/checkpoint") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

# Read from Kafka
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "namenode:9092") \
    .option("subscribe", "test-topic") \
    .option("startingOffsets", "earliest") \
    .option("failOnDataLoss", "false") \
    .load()

# Process messages
df_parsed = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

# Write to console
query = df_parsed.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query.awaitTermination()

:: loading settings :: url = jar:file:/opt/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3cb2f39d-7ac7-4035-bcc3-8b1081c8855a;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
downloading https://repo1.maven.org/maven2/org/apache/spark/spark-sql-kafka-

-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------------+
| key|         value|
+----+--------------+
|NULL|Test message 0|
|NULL|Test message 1|
|NULL|Test message 2|
|NULL|Test message 3|
|NULL|Test message 4|
|NULL|Test message 5|
|NULL|Test message 6|
|NULL|Test message 7|
|NULL|Test message 8|
|NULL|Test message 9|
+----+--------------+

